<a href="https://colab.research.google.com/github/ShoyebKhan09/Action_Detection/blob/main/Object_Detection_tensorflow_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
#!pip install --upgrade tensorflow

In [53]:
import tensorflow as tf
import tensorflow_hub as hub

In [4]:
import random
import re
import os
import tempfile
import ssl
import cv2
import numpy as np

In [5]:
import imageio
from urllib import request

In [55]:
KINETICS_URL = "https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map.txt"
with request.urlopen(KINETICS_URL) as obj:
  labels = [line.decode("utf-8").strip() for line in obj.readlines()]
print("Found %d labels." % len(labels))

Found 400 labels.


In [3]:
def crop_center_square(frame):
  y, x = frame.shape[0:2]
  min_dim = min(y, x)
  start_x = (x // 2) - (min_dim // 2)
  start_y = (y // 2) - (min_dim // 2)
  return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

def load_video(path, max_frames=0, resize=(224, 224)):
  cap = cv2.VideoCapture(path)
  frames = []
  try:
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      frame = crop_center_square(frame)
      frame = cv2.resize(frame, resize)
      frame = frame[:, :, [2, 1, 0]]
      frames.append(frame)

      if len(frames) == max_frames:
        break
  finally:
    cap.release()
  return np.array(frames) / 255.0

def to_gif(images):
  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  return imageio.mimsave('./animation.gif', converted_images, fps=25) 

In [ ]:
#https://drive.google.com/file/d/1_Dyx6Eqh-stY_vOPPPI7BIwaXfplrFm9/view?usp=sharing

#https://drive.google.com/file/d/1RBRAdLY5DJNR3ewdqqOxcmZjbUDnhRMD/view?usp=sharing

In [39]:
#! gdown --id 1bbvRGtPhCABWWq0cSOntDc54e4aF774V

#! gdown --id 1_Dyx6Eqh-stY_vOPPPI7BIwaXfplrFm9

#! gdown --id 1RBRAdLY5DJNR3ewdqqOxcmZjbUDnhRMD

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1_Dyx6Eqh-stY_vOPPPI7BIwaXfplrFm9
To: /content/VID-20220910-WA0002_pru.mp4
100% 2.30M/2.30M [00:00<00:00, 197MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1RBRAdLY5DJNR3ewdqqOxcmZjbUDnhRMD
To: /content/VID-20220910-WA0001_prt.mp4
100% 2.11M/2.11M [00:00<00:00, 9.98MB/s]


In [62]:
#sample_video = '/content/VID-20220909-WA0001.mp4'

#sample_video = '/content/VID-20220910-WA0001_prt.mp4'

sample_video = '/content/VID-20220910-WA0002_pru.mp4'

In [41]:
video_file = load_video(sample_video)

In [42]:
video_file.shape

(243, 224, 224, 3)

In [45]:
i3d = hub.load("https://tfhub.dev/deepmind/i3d-kinetics-400/1").signatures['default']

Using /tmp/tfhub_modules to cache modules.
Downloaded https://tfhub.dev/deepmind/i3d-kinetics-400/1, Total size: 50.35MB
Downloaded TF-Hub Module 'https://tfhub.dev/deepmind/i3d-kinetics-400/1'.


In [46]:
def predict(sample_video):
  # Add a batch axis to the sample video.
  model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]

  logits = i3d(model_input)['default'][0]
  probabilities = tf.nn.softmax(logits)

  print("Top 5 actions:")
  for i in np.argsort(probabilities)[::-1][:5]:
    print(f"  {labels[i]:22}: {probabilities[i] * 100:5.2f}%")

In [47]:
from IPython.display import HTML
from base64 import b64encode

In [48]:
mp4 = open(sample_video,'rb').read()

In [49]:
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [56]:
predict(video_file)

Top 5 actions:
  shooting basketball   : 89.17%
  playing basketball    : 10.58%
  dunking basketball    :  0.20%
  dribbling basketball  :  0.06%
  passing American football (not in game):  0.00%


In [ ]:
## Object Detection on Still Images

In [ ]:
#! gdown --id 1h7GnNA2mvTErzcVXk3DXxrDcWAcFK86h

In [ ]:
input='/content/basketball.png'

In [ ]:
import tensorflow as tf

import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time
import os

In [ ]:
plt.rcParams.update({'figure.max_open_warning': 0})

In [ ]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)
  plt.show()
  plt.savefig('Labelled_Image.png', dpi=100)

def download_and_resize_image(url, new_width=256, new_height=256,
                              display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  #response = urlopen(url)
  #image_data = response.read()
  #image_data = BytesIO(url)
  pil_image = Image.open(url)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename

def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin

def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1):
  """Overlay labeled boxes on an image with formatted scores and label names."""
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image

In [ ]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
detector = hub.load(module_handle).signatures['default']

Using /tmp/tfhub_modules to cache modules.
Downloaded https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1, Total size: 273.85MB
Downloaded TF-Hub Module 'https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1'.
Saver not created because there are no variables in the graph to restore


In [ ]:
def run_detector(detector, path):
  img = load_img(path)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()}

  print("Found %d objects." % len(result["detection_scores"]))
  print("Inference time: ", end_time-start_time)

  image_with_boxes = draw_boxes(
      img.numpy(), result["detection_boxes"],
      result["detection_class_entities"], result["detection_scores"])

  display_image(image_with_boxes)
  return

In [ ]:
downloaded_image_path= download_and_resize_image(input,1280, 856, False)

Image downloaded to /tmp/tmpws18fery.jpg.


In [ ]:
run_detector(detector,downloaded_image_path)

Found 100 objects.
Inference time:  32.947344064712524


In [ ]:
#!pip install tensorflow-object-detection-api
#!pip install tensorflow==1.13.1
#2.10.0

In [1]:
### Object Detection in the Video

In [57]:
import torch
import numpy as np
import cv2
from time import time

In [63]:
class ObjectDetection:
    """
    Class implements Yolo5 model to make inferences on a youtube video using Opencv2.
    """

    def __init__(self,out_file="Labeled_Video_1.mp4"):
        """
        Initializes the class with the input video fiel and output file.
        :param input_file_dir: Has to be as video directory,on which prediction is made.
        :param out_file: A valid output file name.
        """
        self._URL = sample_video
        self.model = self.load_model()
        self.classes = self.model.names
        self.out_file = out_file
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def get_video_from_url(self):
        """
        Creates a new video streaming object to extract video frame by frame to make prediction on.
        :return: opencv2 video capture object, with lowest quality frame available for video.
        """
        return cv2.VideoCapture(sample_video)

    def load_model(self):
        """
        Loads Yolo5 model from pytorch hub.
        :return: Trained Pytorch model.
        """
        model = torch.hub.load('ultralytics/yolov5','yolov5s', pretrained=True)
        return model

    def score_frame(self, frame):
        """
        Takes a single frame as input, and scores the frame using yolo5 model.
        :param frame: input frame in numpy/list/tuple format.
        :return: Labels and Coordinates of objects detected by model in the frame.
        """
        self.model.to(self.device)
        frame = [frame]
        results = self.model(frame)
        labels,cord = results.xyxyn[0][:, -1].numpy(), results.xyxyn[0][:, :-1].numpy()
        return labels,cord

    def class_to_label(self, x):
        """
        For a given label value, return corresponding string label.
        :param x: numeric label
        :return: corresponding string label
        """
        return self.classes[int(x)]

    def plot_boxes(self,results, frame):
        """
        Takes a frame and its results as input, and plots the bounding boxes and label on to the frame.
        :param results: contains labels and coordinates predicted by model on the given frame.
        :param frame: Frame which has been scored.
        :return: Frame with bounding boxes and labels ploted on it.
        """
        labels, cord = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
            row = cord[i]
            if row[4] >= 0.2:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                bgr = (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
                cv2.putText(frame, self.class_to_label(labels[i]), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)

        return frame

    def __call__(self):
        """
        This function is called when class is executed, it runs the loop to read the video frame by frame,
        and write the output into a new file.
        :return: void
        """
        cap     = cv2.VideoCapture(sample_video)
        player  =  cap
        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        four_cc = cv2.VideoWriter_fourcc(*"MJPG")
        out = cv2.VideoWriter(self.out_file, four_cc, 20, (x_shape, y_shape))
        i=0 
        while (i < 1000 ):
            start_time = time()
            ret, frame = cap.read()
            i = i +1 
            if frame is not None: 
              results = self.score_frame(frame)
              frame = self.plot_boxes(results,frame)
              end_time = time()
              fps = 1/np.round(end_time - start_time, 3)
              out.write(frame)
            
# Create a new object and execute.
video_out_labelled = ObjectDetection()
video_out_labelled()  

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-9-15 Python-3.7.13 torch-1.12.1+cu113 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [69]:
output_video = '/content/Labeled_Video_1.mp4'

In [70]:
mp4 = open(output_video,'rb').read()